In [ ]:
#import the libraries
import matplotlib.pyplot as plt
import numpy as np
import os
#import keras
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#image data generator is used to augment the data and preprocess the images 
train_datagener = ImageDataGenerator(
    rescale=1./255,#noormalizes the images to the range [0,1]
    shear_range = 0.2,#this distrots the shape of the image along a specified axis, creating a slanting effect.
    zoom_range = 0.2,#randomly zooms in on the image creating a zooming effect.
    horizontal_flip=True#randomnly flips the image horizontally
    )

val_datagen = ImageDataGenerator(rescale=1./255)#normalize the images of the validation data

#this function loads the data from the directory and applies the transformation on the dataset
train_ds = train_datagener.flow_from_directory(
    directory = "wheat_disease/data/train",
    target_size=(256,256),
    #batch_size=32,
    class_mode='categorical',
)

val_ds = val_datagen.flow_from_directory(
    directory = "wheat_disease/data/test",
    target_size=(256,256),
    #batch_size=32,
    class_mode='categorical'
)

In [ ]:

from keras.regularizers import l2

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),input_shape=(256,256,3),activation='relu',kernel_regularizer=l2(0.0002),padding='same'))
layers.BatchNormalization()
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(32,(3,3),activation='relu',kernel_regularizer=l2(0.0002),padding='same'))
layers.BatchNormalization()
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(32,(3,3),activation='relu',kernel_regularizer=l2(0.0002),padding='same'))
model.add(layers.Dropout(0.3))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(32,(3,3),activation='relu',kernel_regularizer=l2(0.0002),padding='same'))
model.add(layers.Dropout(0.3))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(15,activation='softmax'))

model.summary()

In [ ]:
#compile the model
from tensorflow.keras import callbacks
model.compile(optimizer='adam',
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = ['accuracy'])

lr_schedule = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4)
#history = model.fit(train_ds, epochs=20, validation_data=val_ds,batch_size=32,validation_batch_size=32)
model_checkpoint = callbacks.ModelCheckpoint(f'wheat best_model accuracy.keras', save_best_only=True, monitor='val_loss')
history = model.fit(train_ds, epochs=70, validation_data=val_ds,batch_size=20,callbacks=[model_checkpoint,lr_schedule])


In [ ]:
fig = plt.figure(figsize=(14,7))

fig.add_subplot(121)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Training accuracy', 'validation accuracy'])
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

fig.add_subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training loss', 'validation loss'])
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
import numpy as np
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define class labels
wheat_class_labels = ['Aphid', 'Black Rust', 'Blast', 'Brown Rust', 'Common Root Rot', 'Fusarium Head Blight', 'Healthy', 'Leaf Blight', 'Mildew', 'Mite', 'Septoria', 'Smut', 'Stem Fly', 'Tan Spot', 'Yellow Rust']

# Load the model
model_path = "wheat_disease_model accuracy 0.8933.keras"
model = load_model(model_path)
print("Model loaded successfully.")

# Define the test data directory
test_data_dir = "wheat_disease/data/test"

# Create an ImageDataGenerator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a test data generator
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(255, 255),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Predict on the test data
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Print the classification report
from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

In [ ]:
import numpy as np
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# Define class labels
wheat_class_labels = ['Aphid', 'Black Rust', 'Blast', 'Brown Rust', 'Common Root Rot', 'Fusarium Head Blight', 'Healthy', 'Leaf Blight', 'Mildew', 'Mite', 'Septoria', 'Smut', 'Stem Fly', 'Tan Spot', 'Yellow Rust']

# Load the model
model_path = "wheat_disease_model accuracy 0.8933.keras"
model = load_model(model_path)
print("Model loaded successfully.")

# Define the test data directory
test_data_dir = "wheat_disease/data/test"

# Create an ImageDataGenerator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a test data generator
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(255, 255),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Predict on the test data
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Print the classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()